# Chapter 09-03: 파이프라인 병렬화 (Pipeline Parallelism)

## 학습 목표
- 파이프라인 병렬화가 필요한 상황을 이해하고, Naive 방식의 **Bubble 시간**을 수식으로 정량화한다
- GPipe 방식에서 마이크로배치 수 $m$이 Bubble을 $\frac{p-1}{m+p-1}$로 줄이는 원리를 증명한다
- **1F1B(One Forward One Backward)** 스케줄이 GPipe 대비 **메모리**를 $O(p)$로 줄이는 메커니즘을 이해한다
- 파이프라인 스케줄을 타임라인 다이어그램으로 시각화하고 실제 활용률을 분석한다

## 목차
1. [수학적 기초: Bubble 비율 공식](#1.-수학적-기초)
2. [파이프라인 병렬화 개념](#2.-파이프라인-병렬화-개념)
3. [Naive Pipeline: Bubble 정량 분석](#3.-Naive-Pipeline)
4. [GPipe: 마이크로배치 해법](#4.-GPipe)
5. [1F1B: 메모리 최적화 스케줄](#5.-1F1B-스케줄)
6. [종합 비교 시각화](#6.-종합-비교)
7. [정리 및 연습 문제](#7.-정리)

## 1. 수학적 기초

### 파이프라인 Bubble 비율 공식

| 방식 | Bubble 비율 | GPU 활용률 |
|------|------------|----------|
| Naive ($m=1$) | $\displaystyle\frac{p-1}{p}$ | $\displaystyle\frac{1}{p}$ |
| GPipe ($m$개 마이크로배치) | $\displaystyle\frac{p-1}{m+p-1}$ | $\displaystyle\frac{m}{m+p-1}$ |
| 이상적 ($m \to \infty$) | $0$ | $1$ (100%) |

여기서 $p$ = 파이프라인 스테이지(GPU) 수, $m$ = 마이크로배치 수.

### GPipe vs 1F1B Peak Activation Memory

$$M_{peak}^{\text{GPipe}} \propto m \cdot M_{act, 1}$$
$$M_{peak}^{\text{1F1B}} \propto p \cdot M_{act, 1}$$

- $M_{act, 1}$: 마이크로배치 1개의 Activation Memory
- GPipe: 모든 Forward를 다 끝내고 Backward 시작 → $m$개 Activation 동시 보유
- 1F1B: Forward 1개마다 Backward 1개 바로 수행 → 최대 $p$개 동시 보유
- $m \gg p$인 경우 **1F1B가 훨씬 메모리 효율적**

---

### 🐣 초등학생을 위한 친절 설명!

#### 🏭 파이프라인 병렬화가 뭔가요?

> 자동차 공장의 조립 라인을 상상해 보세요:
> ```
> 작업자 1 (GPU 0): 차체 조립      → 다음으로 전달
> 작업자 2 (GPU 1): 엔진 장착      → 다음으로 전달  
> 작업자 3 (GPU 2): 도장            → 다음으로 전달
> 작업자 4 (GPU 3): 검수            → 완성!
> ```
> 한 대가 작업자 1을 떠나면 바로 다음 차가 들어옵니다.
> 하지만 처음에 채우는 시간과 끝에 비우는 시간이 낭비돼요 → 이것이 **Bubble!**

#### 💡 GPipe는 어떻게 Bubble을 줄이나요?

> 차 1대를 한 번에 만들면? → Bubble이 큼  
> 차를 4개의 작은 조각(마이크로배치)으로 나눠서 연속 투입? → Bubble이 작아짐!

### 📝 연습 문제

#### 문제 1: GPipe Bubble 비율 계산

GPU 8대로 파이프라인 구성, 마이크로배치 32개 사용 시 GPU 활용률은?

<details>
<summary>💡 풀이 확인</summary>

$p=8$, $m=32$

$$\text{Bubble 비율} = \frac{p-1}{m+p-1} = \frac{7}{32+8-1} = \frac{7}{39} \approx 17.9\%$$

$$\text{GPU 활용률} = 1 - 17.9\% = \mathbf{82.1\%}$$
</details>

#### 문제 2: 활용률 90% 달성을 위한 $m$ 최솟값

$p=4$일 때, Bubble 비율 < 10%가 되려면 $m$이 얼마여야 하는가?

<details>
<summary>💡 풀이 확인</summary>

$$\frac{p-1}{m+p-1} < 0.1 \Rightarrow \frac{3}{m+3} < 0.1 \Rightarrow m + 3 > 30 \Rightarrow m > 27$$

최솟값: $m = \mathbf{28}$
</details>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch

np.random.seed(42)

def bubble_ratio(p, m):
    """GPipe Bubble 비율: (p-1)/(m+p-1)"""
    return (p - 1) / (m + p - 1)

def gpu_utilization(p, m):
    return 1 - bubble_ratio(p, m)

print("파이프라인 Bubble 비율 계산 함수 정의 완료")

# 다양한 설정에서의 Bubble 비율 테이블
print("\n=== Bubble 비율 & GPU 활용률 표 ===")
print(f"{'p (GPU수)':>10} | {'m=1':>8} | {'m=4':>8} | {'m=8':>8} | {'m=16':>8} | {'m=32':>8}")
print("-" * 58)
for p in [2, 4, 8, 16]:
    row = f"{p:>10} |"
    for m in [1, 4, 8, 16, 32]:
        util = gpu_utilization(p, m) * 100
        row += f" {util:>6.1f}% |"
    print(row)

## 2. 파이프라인 병렬화 개념

In [ ]:
# ---------------------------------------------------
# 파이프라인 스케줄 타임라인 시각화
# ---------------------------------------------------

def draw_timeline(ax, schedule, title, p, forward_color='#42A5F5', backward_color='#EF5350'):
    """
    파이프라인 스케줄을 타임라인 다이어그램으로 그린다.
    schedule: {(gpu_id, time_step): ('F'|'B'|'W', microbatch_id)}
    """
    if not schedule:
        return

    max_t = max(t for _, t in schedule.keys()) + 1
    n_mb = max(mb for _, (_, mb) in schedule.items()) + 1

    alpha_steps = np.linspace(0.45, 0.9, n_mb)

    total = len(schedule)
    bubble_cells = max_t * p - total

    for (gpu, t), (phase, mb) in schedule.items():
        base = forward_color if phase in ('F',) else backward_color
        alpha = alpha_steps[mb % n_mb]
        rect = FancyBboxPatch((t, p - 1 - gpu), 0.92, 0.82,
                              boxstyle="round,pad=0.04",
                              facecolor=base, edgecolor='white',
                              linewidth=1.5, alpha=alpha)
        ax.add_patch(rect)
        label = f"{phase}{mb}"
        ax.text(t + 0.46, p - 1 - gpu + 0.41, label,
                ha='center', va='center', fontsize=7.5, fontweight='bold', color='white')

    bubble_pct = bubble_cells / (max_t * p) * 100
    util_pct   = 100 - bubble_pct

    ax.set_xlim(-0.3, max_t + 0.3)
    ax.set_ylim(-0.4, p + 0.5)
    ax.set_yticks(range(p))
    ax.set_yticklabels([f'GPU {p-1-i}' for i in range(p)], fontsize=9)
    ax.set_xlabel('Time Step', fontsize=9)
    ax.set_title(f'{title}\nBubble: {bubble_pct:.0f}% | GPU 활용률: {util_pct:.0f}%',
                 fontweight='bold', fontsize=10)
    ax.grid(True, alpha=0.15, lw=0.5)

print("타임라인 시각화 함수 정의 완료")

## 3. Naive Pipeline

In [ ]:
# ---------------------------------------------------
# Naive Pipeline 스케줄 생성 (m=1)
# ---------------------------------------------------

def make_naive_schedule(p):
    """Naive Pipeline (m=1): Forward→Backward 순서"""
    sched = {}
    # Forward: 0,1,...,p-1
    for stage in range(p):
        sched[(stage, stage)] = ('F', 0)
    # Backward: p-1,...,1,0
    for i, stage in enumerate(range(p-1, -1, -1)):
        sched[(stage, p + i)] = ('B', 0)
    return sched

p = 4
naive_sched = make_naive_schedule(p)

print(f"Naive Pipeline (p={p}, m=1)")
print(f"  이론 Bubble 비율: {bubble_ratio(p, 1)*100:.0f}% (GPU 활용률: {gpu_utilization(p,1)*100:.0f}%)")
print(f"  총 타임 스텝: {max(t for _, t in naive_sched.keys())+1}")
print(f"  총 연산 셀: {len(naive_sched)} (Full: {p}×{max(t for _,t in naive_sched.keys())+1}={p*(max(t for _,t in naive_sched.keys())+1)})")

fig, ax = plt.subplots(figsize=(10, 3.5))
draw_timeline(ax, naive_sched, f'Naive Pipeline (p={p}, m=1)', p)

# Bubble 영역 표시
max_t = max(t for _, t in naive_sched.keys()) + 1
for gpu in range(p):
    busy_times = {t for (g, t), _ in naive_sched.items() if g == gpu}
    for t in range(max_t):
        if t not in busy_times:
            rect = FancyBboxPatch((t, p-1-gpu), 0.92, 0.82,
                                  boxstyle="round,pad=0.04",
                                  facecolor='#BDBDBD', edgecolor='white',
                                  linewidth=1, alpha=0.5)
            ax.add_patch(rect)
            ax.text(t+0.46, p-1-gpu+0.41, '💤', ha='center', va='center', fontsize=9)

plt.tight_layout()
plt.show()
print(f"\n→ p={p}에서 GPU 활용률이 25%에 불과! 3/4의 시간 동안 GPU가 놀고 있다.")

## 4. GPipe

In [ ]:
# ---------------------------------------------------
# GPipe 스케줄 (마이크로배치 m개)
# ---------------------------------------------------

def make_gpipe_schedule(p, m):
    """GPipe: 모든 Forward 먼저, 그 다음 모든 Backward"""
    sched = {}
    # 모든 마이크로배치 Forward
    for mb in range(m):
        for stage in range(p):
            t = mb + stage
            sched[(stage, t)] = ('F', mb)
    # 모든 마이크로배치 Backward (역순)
    fwd_end = m + p - 1
    for mb in range(m):
        for i, stage in enumerate(range(p-1, -1, -1)):
            t = fwd_end + mb * p + i
            sched[(stage, t)] = ('B', mb)
    return sched


m = 4
gpipe_sched = make_gpipe_schedule(p, m)

print(f"GPipe (p={p}, m={m})")
print(f"  이론 Bubble 비율: {bubble_ratio(p,m)*100:.1f}% (활용률: {gpu_utilization(p,m)*100:.1f}%)")
print(f"  메모리 비용: m={m}개 Activation 동시 보유 (GPipe의 단점!)")

fig, ax = plt.subplots(figsize=(16, 3.5))
draw_timeline(ax, gpipe_sched, f'GPipe (p={p}, m={m})', p)
plt.tight_layout()
plt.show()

## 5. 1F1B 스케줄

1F1B의 핵심은 **Warmup 단계**에서 Forward만 수행하여 파이프라인을 채운 뒤,  
**Steady 단계**에서 Forward 1개 완료 즉시 Backward 1개를 수행하는 것입니다.

```
시간 →   0  1  2  3  4  5  6  7  8  9 10 11
GPU 0:  F0 F1 F2 F3 B0 F4 B1 F5 B2 F6 B3  B$
GPU 1:     F0 F1 F2 F3 B0 F4 B1 F5 B2 F6  B$
GPU 2:        F0 F1 F2 F3 B0 F4 B1 F5     B$
GPU 3:           F0 F1 F2 F3 B0 F4        B$
                 ↑── Warm ──↑───Steady───↑─Cool
```

**Peak Activation Memory**: Steady 단계에서 최대 **p개**의 Forward Activation만 보유  
(GPipe는 **m개** 보유 → m=32, p=8이면 4배 절약!)

In [ ]:
# ---------------------------------------------------
# 1F1B 스케줄 근사 구현
# ---------------------------------------------------

def make_1f1b_schedule_approx(p, m):
    """
    1F1B 스케줄 근사 구현.
    실제 Megatron-LM 구현은 더 복잡하지만, 개념 설명용으로 단순화.
    """
    sched = {}
    # Warmup: 파이프라인 채우기 (각 스테이지가 자신 차례까지 Forward)
    for stage in range(p):
        for mb in range(min(p - stage, m)):
            t = mb + stage
            if (stage, t) not in sched:
                sched[(stage, t)] = ('F', mb)

    # Steady + Cooldown: 1F1B 패턴 (간략화)
    # GPU 0: Fwd mb 계속 진행, Bwd는 Fwd w/ offset
    t_offset = p - 1
    for mb in range(m):
        for stage in range(p):
            fwd_t = mb + stage
            bwd_t = mb + 2*p - 1 - stage + (p - 1)
            if mb < m and (stage, bwd_t) not in sched:
                sched[(stage, bwd_t)] = ('B', mb)

    return sched


m_1f1b = 8  # 마이크로배치 수를 늘려 비교
gpipe_8 = make_gpipe_schedule(p, m_1f1b)

print(f"=== Bubble 비율 비교 (p={p}) ===")
for m_val in [1, 2, 4, 8, 16, 32]:
    b = bubble_ratio(p, m_val)
    print(f"  m={m_val:>3}: Bubble={b*100:>5.1f}%, 활용률={100-b*100:>5.1f}%",
          " ← GPipe/1F1B 동일" if m_val > 1 else " ← Naive")

# GPipe m=8 시각화
fig, ax = plt.subplots(figsize=(18, 3.5))
draw_timeline(ax, gpipe_8, f'GPipe (p={p}, m={m_1f1b})', p)
plt.tight_layout()
plt.show()

print(f"\n1F1B의 핵심 장점:")
print(f"  - GPipe m={m_1f1b}과 동일한 Bubble 비율 ({bubble_ratio(p,m_1f1b)*100:.1f}%)")
print(f"  - 하지만 Peak Activation Memory: GPipe={m_1f1b}× vs 1F1B={p}× (절약 {m_1f1b//p}배!)")

## 6. 종합 비교 시각화

In [ ]:
# ---------------------------------------------------
# Bubble 비율 & 메모리 비교 종합 시각화
# ---------------------------------------------------

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ── 왼쪽: m에 따른 Bubble 비율 변화 ──────────────────
ax1 = axes[0]
m_range = np.arange(1, 65)
colors = ['#1E88E5', '#E53935', '#43A047', '#FB8C00']
for p_val, c in zip([2, 4, 8, 16], colors):
    bubble = [bubble_ratio(p_val, m) * 100 for m in m_range]
    ax1.plot(m_range, bubble, '-', color=c, lw=2,
             label=f'p={p_val} GPU')  

ax1.axhline(y=10, color='k', ls='--', lw=1.5, alpha=0.6, label='10% 목표선')
ax1.fill_between(m_range, 0, 10, alpha=0.05, color='green')
ax1.text(55, 4, '목표 구간', color='green', fontsize=9)
ax1.set_xlabel('마이크로배치 수 (m)', fontsize=11)
ax1.set_ylabel('Bubble 비율 (%)', fontsize=11)
ax1.set_title('GPipe Bubble 비율\nm이 클수록 감소', fontweight='bold')
ax1.legend(fontsize=9); ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 90)

# ── 오른쪽: GPipe vs 1F1B 메모리 비교 ──────────────────
ax2 = axes[1]
m_range2 = np.arange(1, 33)
p_fixed = 4

gpipe_mem = m_range2   # ∝ m
onefb_mem = np.full_like(m_range2, p_fixed, dtype=float)  # ∝ p (상수)

ax2.plot(m_range2, gpipe_mem, 'r-', lw=2.5, label=f'GPipe: Memory ∝ m')
ax2.plot(m_range2, onefb_mem, 'b-', lw=2.5, label=f'1F1B: Memory ∝ p={p_fixed} (상수!)')
ax2.fill_between(m_range2, onefb_mem, gpipe_mem,
                 alpha=0.12, color='red', label='1F1B 절약 구간')

# 특정 m에서 절약량 표시
for m_show in [8, 16, 32]:
    save = m_show - p_fixed
    ax2.annotate(f'm={m_show}: 1F1B\n {m_show//p_fixed}배 적음',
                 xy=(m_show, (m_show + p_fixed) / 2),
                 fontsize=8, color='red', ha='left')

ax2.set_xlabel('마이크로배치 수 (m)', fontsize=11)
ax2.set_ylabel('Peak Activation Memory (상대값)', fontsize=11)
ax2.set_title(f'GPipe vs 1F1B Peak Memory\n(p={p_fixed})', fontweight='bold')
ax2.legend(fontsize=9); ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("결론:")
print(f"  • GPipe: m 늘릴수록 Bubble 감소 → 좋음. 하지만 Memory도 m배 증가 → 나쁨")
print(f"  • 1F1B: 동일한 Bubble 비율에서 Memory를 p={p_fixed}로 고정 → GPipe의 메모리 단점 해결!")
print(f"  • 실무(Megatron-LM, DeepSpeed): 1F1B가 표준 방식")

In [ ]:
# ---------------------------------------------------
# 실제 대규모 학습 설정 사례 분석
# ---------------------------------------------------

print("=== 실제 대규모 학습 Pipeline Parallelism 설정 ===")
print(f"\n{'모델':<25} {'크기':>8} | {'TP':>4} | {'PP':>4} | {'DP':>5} | {'총GPU':>6} | {'마이크로배치':>10}")
print("-" * 75)

configs = [
    ('GPT-3',           '175B', 8,   8,  64,  4096, 32),
    ('Llama-2 70B',     '70B',  8,   4,  16,   512, 16),
    ('Megatron-Turing', '530B', 8,  35,   6,  4480, 8),
    ('DeepSeek-V3',     '671B', 8,  16,  var := 48, 6144, 'large'),
]

configs = [
    ('GPT-3',           '175B', 8,   8,  64,  4096, 32),
    ('Llama-2 70B',     '70B',  8,   4,  16,   512, 16),
    ('Megatron-Turing', '530B', 8,  35,   6,  4480,  8),
]

for name, size, tp, pp, dp, n_gpu, m in configs:
    bub = bubble_ratio(pp, m) * 100
    print(f"{name:<25} {size:>8} | {tp:>4} | {pp:>4} | {dp:>5} | {n_gpu:>6} |"
          f" m={m} (Bubble={bub:.0f}%)")

print("\n핵심 패턴:")
print("  • TP는 항상 ≤ 노드 내 GPU 수 (8)  ← NVLink로 연결된 범위 내")
print("  • PP는 노드 간 연결 기반           ← InfiniBand로 스테이지 간 통신")
print("  • DP는 가장 바깥쪽 루프             ← 전체 그래디언트 동기화")

## 7. 정리

### 핵심 개념 요약

| 방식 | Bubble 비율 | Peak Memory | 구현 복잡도 |
|------|------------|------------|----------|
| **Naive** ($m=1$) | $\frac{p-1}{p}$ → 크다 | $O(1)$ | 매우 간단 |
| **GPipe** | $\frac{p-1}{m+p-1}$ → 작다 | $O(m)$ | 보통 |
| **1F1B** | $\frac{p-1}{m+p-1}$ → 동일 | $O(p)$ | 복잡 |

### 핵심 수식

$$\text{Bubble}_{\text{GPipe}} = \frac{p-1}{m+p-1} \xrightarrow{m \gg p} 0$$

$$M_{peak}^{\text{GPipe}} = m \cdot M_{act,1} \quad \text{vs} \quad M_{peak}^{\text{1F1B}} = p \cdot M_{act,1}$$

### 왜 1F1B가 실무 표준인가?
- **Bubble 비율**: GPipe와 동일 (마이크로배치 수 $m$으로 제어)
- **메모리**: $m$이 커져도 $p$에 고정돼 있어 훨씬 효율적
- **구현**: Megatron-LM, DeepSpeed 모두 1F1B 지원

### 다음 챕터 예고
**Chapter 09-04: 3D 병렬화** — DP+TP+PP를 하나의 통합된 시스템으로 결합하는 3D Parallelism 설계 원칙, 하드웨어 토폴로지에 맞는 최적 배치 전략, 실제 대규모 모델 훈련 사례를 분석한다.